In [ ]:
# convert into document vocab matrix


In [3]:
import pandas as pd
from ast import literal_eval
import gc
from sklearn.feature_extraction.text import CountVectorizer

path = "../clean_data/lemmatized_tokenized_data.csv"

col = "tokenized_text"

df = pd.read_csv(path, usecols=[col])
    
df.head() 

,tokenized_text
0,"['last', 'federal', 'reserve', 'board', 'issue..."
1,"['test', 'door', 'service', 'service', 'board'..."
2,"['sanction', 'chinese', 'contract']"
3,"['lead', 'frazier', '4', '496', 'langer', 'mar..."
4,"['chicago', 'april', '30', '300', 'suspicious'..."


In [ ]:

df[col] = df[col].apply(lambda x: " ".join(literal_eval(x)))
df.head()

,tokenized_text
0,last federal reserve board issued rule organiz...
1,test door service service board examine r safe...
2,sanction chinese contract
3,lead frazier 4 496 langer margin fall north da...
4,chicago april 30 300 suspicious character incl...


In [5]:
vectorizer = CountVectorizer()

data = vectorizer.fit_transform(df[col])

In [8]:
data.shape

(11027532, 1328101)

In [10]:
from scipy import sparse
import numpy as np

def save_sparse_csr(filename, array):
    np.savez(filename, data=array.data, indices=array.indices,
             indptr=array.indptr, shape=array.shape)
    
save_sparse_csr("dtm.npz", data)

In [12]:
ndata = np.load("dtm.npz")

In [16]:
del df
gc.collect()

5580

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
import gc
N_TOPICS = 5
chunk_size = 100000

lda = LatentDirichletAllocation(
    n_components=N_TOPICS,
    learning_method='online',  # Allows partial_fit
    random_state=42,
    batch_size=chunk_size,
    max_iter=1,                  # One pass per chunk,
    n_jobs=-1
)

fitted = lda.fit(data)
#for dtm in pd.read_csv("dtm.csv", chunk_size):
#    lda.partial_fit(dtm)    
#    del dtm
#    gc.collect()


In [ ]:
def print_top_words(model, feature_names, n_top_words=10):
    for topic_idx, topic in enumerate(model.components_):
        top_features = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        print(f"Topic #{topic_idx+1}: {' | '.join(top_features)}")

print_top_words(lda, vectorizer.get_feature_names_out())

In [ ]:
import pickle

pickle.dump(lda, 'lda_model.pk')
